In [1]:
#Import all necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
retaildf = pd.read_csv('/Users/snehakarki/Downloads/RetailProj/new_retail_data.csv')
retaildf

#302010 rows × 30 columns

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302005,4246475.0,12104.0,Meagan Ellis,Courtney60@gmail.com,7.466354e+09,389 Todd Path Apt. 159,Townsville,New South Wales,4567.0,Australia,...,973.962984,Books,Penguin Books,Fiction,Bad,Same-Day,Cash,Processing,1.0,Historical fiction
302006,1197603.0,69772.0,Mathew Beck,Jennifer71@gmail.com,5.754305e+09,52809 Mark Forges,Hanover,Berlin,16852.0,Germany,...,285.137301,Electronics,Apple,Laptop,Excellent,Same-Day,Cash,Processing,5.0,LG Gram
302007,7743242.0,28449.0,Daniel Lee,Christopher100@gmail.com,9.382530e+09,407 Aaron Crossing Suite 495,Brighton,England,88038.0,UK,...,182.105285,Clothing,Adidas,Jacket,Average,Express,Cash,Shipped,2.0,Parka
302008,9301950.0,45477.0,Patrick Wilson,Rebecca65@gmail.com,9.373222e+09,3204 Baird Port,Halifax,Ontario,67608.0,Canada,...,120.834784,Home Decor,IKEA,Furniture,Good,Standard,Cash,Shipped,4.0,TV stand


- Will start by selecting those columns that are relevant to the project and problem scope. After that will handle null values, perform any encoding if reqd, datatype conversion.
- This will the be followed by more EDA - need for feature engineering, feature selection, standardization or normalization before diving deeper into model building and testing!
- eg: demograph_df = retaildf[['Age', 'Gender', 'Income','Transaction_ID','Customer_ID','Country', 'Customer_Segment', 'Total_Purchases', 'Amount','Total_Amount']]

demograph_df.head()

## Data Cleaning
1. Null Values
2. Erroneous Data
3. Data Type Conversion Required? 
4. Removing certain columns if reqd?

#### Coming back after first round of analysis - Look at it from a business point of view!

1. Do I need to do PCA or dimensionality reduction?
2. Feature Scaling?
3. Feature Engineering?

In [3]:
missing_percentage = (retaildf.isnull().sum()/len(retaildf))*100
missing_percentage

distinct_entries = retaildf.nunique().sort_values(ascending=False)
print(distinct_entries)

Address             299329
Total_Amount        299305
Amount              299297
Phone               299295
Transaction_ID      294461
Name                159390
Zipcode              93978
Customer_ID          86766
Time                 83677
Email                52897
Date                   366
products               318
City                   130
State                   54
Age                     53
Product_Type            33
Product_Brand           18
Month                   12
Total_Purchases         10
Product_Category         5
Ratings                  5
Country                  5
Payment_Method           4
Order_Status             4
Feedback                 4
Income                   3
Shipping_Method          3
Customer_Segment         3
Gender                   2
Year                     2
dtype: int64


 The percentage of missing values is very, very low compared to the total number of rows.
At this point it would be right to simply remove them

Columns thatshould be deletd - Transaction ID, 


In [4]:
columns_with_na = ['Address','Date','Total_Amount','Amount','Transaction_ID','Customer_ID','Name','Email','Phone','Zipcode']

retaildf.dropna(subset=columns_with_na, inplace=True)

#Importing the reqd library, pacakage for imputation
from sklearn.impute import SimpleImputer

#Imputing Columns 

# Fill NaN with 0 for 'Date'
retaildf['Date'].fillna('0', inplace=True)

# Impute based on related columns
# Impute 'Products' based on 'Product_Category'
# to fill missing values in the products column based on the most frequent products value within each Product_Category
retaildf['products'].fillna(retaildf.groupby('Product_Category')['products'].transform(lambda x: x.mode()[0]), inplace=True)

# Impute 'City' based on 'State'
retaildf['City'].fillna(retaildf.groupby('State')['City'].transform(lambda x: x.mode()[0]), inplace=True)

# Impute 'State' based on 'Country'
retaildf['State'].fillna(retaildf.groupby('Country')['State'].transform(lambda x: x.mode()[0]), inplace = True)

# Impute 'Product_Type' based on 'Product_Brand'
retaildf['Product_Type'].fillna(retaildf.groupby('Product_Brand')['Product_Type'].transform(lambda x: x.mode()[0]), inplace=True)

# Impute 'Product_Brand' based on 'Product_Type'
retaildf['Product_Brand'].fillna(retaildf.groupby('Product_Type')['Product_Brand'].transform(lambda x: x.mode()[0]), inplace=True)

#Imputer 'Month' based on 'Date' if it isn't null otherwise by the most popular month
popular_month = retaildf['Month'].mode()[0]
retaildf['Month'] = retaildf.apply(lambda row: row['Month'] if pd.notnull(row['Date']) else popular_month, axis=1)




In [5]:
#To impute non-numeric, categorical data from the rest of the columns ↓ 

numerical_columns = ['Age', 'Total_Purchases', 'Ratings']
mean_imputer = SimpleImputer(strategy='mean')
retaildf[numerical_columns] = mean_imputer.fit_transform(retaildf[numerical_columns])

# Impute categorical columns with mode
categorical_columns = ['Product_Category', 'Country', 'Payment_Method', 
                       'Order_Status', 'Feedback', 'Shipping_Method', 
                       'Customer_Segment', 'Gender', 'Income']

for col in categorical_columns:
    retaildf[col].fillna(retaildf[col].mode()[0], inplace=True)

new_missing_percentage = (retaildf.isnull().sum()/len(retaildf))*100
new_missing_percentage

Transaction_ID      0.000000
Customer_ID         0.000000
Name                0.000000
Email               0.000000
Phone               0.000000
Address             0.000000
City                0.000000
State               0.000335
Zipcode             0.000000
Country             0.000000
Age                 0.000000
Gender              0.000000
Income              0.000000
Customer_Segment    0.000000
Date                0.000000
Year                0.116553
Month               0.090429
Time                0.115883
Total_Purchases     0.000000
Amount              0.000000
Total_Amount        0.000000
Product_Category    0.000000
Product_Brand       0.000000
Product_Type        0.000000
Feedback            0.000000
Shipping_Method     0.000000
Payment_Method      0.000000
Order_Status        0.000000
Ratings             0.000000
products            0.000000
dtype: float64

We've handled the missing values fairly well, and whatever is still left we can drop it since it is a very small portion of the dataset.

In [6]:
'''state_mode = retaildf['State'].mode()[0]
retaildf['State'].fillna(state_mode, inplace=True)

income_mode = retaildf['Income'].mode()[0]
retaildf['Income'].fillna(income_mode, inplace=True)
''' 
year_mode = retaildf['Year'].mode()[0]
retaildf['Year'].fillna(year_mode, inplace=True)

time_mode = retaildf['Time'].mode()[0]
retaildf['Time'].fillna(time_mode, inplace=True)

month_mode = retaildf['Month'].mode()[0]
retaildf['Month'].fillna(month_mode, inplace=True)

retaildf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298577 entries, 0 to 302009
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    298577 non-null  float64
 1   Customer_ID       298577 non-null  float64
 2   Name              298577 non-null  object 
 3   Email             298577 non-null  object 
 4   Phone             298577 non-null  float64
 5   Address           298577 non-null  object 
 6   City              298577 non-null  object 
 7   State             298576 non-null  object 
 8   Zipcode           298577 non-null  float64
 9   Country           298577 non-null  object 
 10  Age               298577 non-null  float64
 11  Gender            298577 non-null  object 
 12  Income            298577 non-null  object 
 13  Customer_Segment  298577 non-null  object 
 14  Date              298577 non-null  object 
 15  Year              298577 non-null  float64
 16  Month             29

In [7]:
import datetime

# List of columns to convert to int
columns_to_int = ['Transaction_ID', 'Customer_ID', 'Phone', 'Zipcode', 'Age', 'Total_Purchases', 'Ratings']

# Convert the columns to int
for column in columns_to_int:
    retaildf[column] = retaildf[column].astype(int)

retaildf['Total_Amount'] = retaildf['Total_Amount'].astype(float).round(2)
retaildf['Amount'] = retaildf['Amount'].astype(float).round(2)

#time related columns
retaildf['Year'] = retaildf['Year'].astype(int)
#retaildf['Month'] = retaildf['Month'].astype(int) - string values: Jan - Dec

retaildf['Date'] = pd.to_datetime(retaildf['Date'])
#retaildf['Time'] = pd.to_timedelta(retaildf['Time'])

# Display the first few rows to check the conversion
retaildf.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788,37249,Michelle Harrington,Ebony39@gmail.com,1414786801,3959 Amanda Burgs,Dortmund,Berlin,77985,Germany,...,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5,Cycling shorts
1,2174773,69749,Kelsey Hill,Mark36@gmail.com,6852899987,82072 Dawn Centers,Nottingham,England,99071,UK,...,806.71,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4,Lenovo Tab
2,6679610,30192,Scott Jensen,Shane85@gmail.com,8362160449,4133 Young Canyon,Geelong,New South Wales,75929,Australia,...,1063.43,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2,Sports equipment
3,7232460,62101,Joseph Miller,Mary34@gmail.com,2776751724,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420,Canada,...,2466.85,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4,Utility knife
4,4983775,27901,Debra Coleman,Charles30@gmail.com,9098267635,5813 Lori Ports Suite 269,Bristol,England,48704,UK,...,248.55,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1,Chocolate cookies


In [8]:
#Noticed some of the brands were placed under an incorrect category
category_brand_type =  retaildf.groupby(['Product_Category', 'Product_Brand'])['Product_Type'].unique().reset_index()

# Display the results
print(category_brand_type)

# To get a count of product types for each category and brand
category_brand_type_count = retaildf.groupby(['Product_Category', 'Product_Brand', 'Product_Type']).size().reset_index(name='Count')

print("\nDetailed view with counts:")

   Product_Category      Product_Brand                          Product_Type
0             Books      HarperCollins      [Non-Fiction, Thriller, Fiction]
1             Books      Penguin Books    [Children's, Non-Fiction, Fiction]
2             Books       Random House    [Non-Fiction, Literature, Fiction]
3          Clothing             Adidas              [T-shirt, Shoes, Jacket]
4          Clothing               Nike              [Shorts, Shoes, T-shirt]
5          Clothing               Zara                 [Shirt, Dress, Jeans]
6       Electronics             Adidas              [Jacket, Shoes, T-shirt]
7       Electronics              Apple          [Tablet, Laptop, Smartphone]
8       Electronics  Bed Bath & Beyond          [Bathroom, Bedding, Kitchen]
9       Electronics           BlueStar                         [BlueStar AC]
10      Electronics          Coca-Cola            [Juice, Water, Soft Drink]
11      Electronics      HarperCollins      [Non-Fiction, Fiction, Thriller]

In [9]:
brand_category_map = {
    'Adidas': 'Clothing',
    'Nike': 'Clothing',
    'Zara': 'Clothing'
}

# Function to correct the category
def correct_category(row):
    if row['Product_Brand'] in brand_category_map:
        return brand_category_map[row['Product_Brand']]
    return row['Product_Category']

# Apply the correction
retaildf['Corrected_Category'] = retaildf.apply(correct_category, axis=1)

# Display the rows where corrections were made
corrections = retaildf[retaildf['Product_Category'] != retaildf['Corrected_Category']]

# Update the original Product_Category column
retaildf['Product_Category'] = retaildf['Corrected_Category']

# Remove the temporary Corrected_Category column
retaildf = retaildf.drop('Corrected_Category', axis=1)

# Verify the changes
print("\nUpdated categorizations:")
print(retaildf[retaildf['Product_Brand'].isin(['Adidas', 'Nike', 'Zara'])][['Product_Category', 'Product_Brand', 'Product_Type']])

# Count of corrected entries
corrected_count = len(corrections)
print(f"\nNumber of entries corrected: {corrected_count}")


Updated categorizations:
       Product_Category Product_Brand Product_Type
0              Clothing          Nike       Shorts
7              Clothing          Zara        Shirt
14             Clothing        Adidas      T-shirt
30             Clothing        Adidas        Shoes
36             Clothing          Zara        Shirt
...                 ...           ...          ...
301983         Clothing          Nike      T-shirt
301988         Clothing          Zara        Dress
301995         Clothing        Adidas      T-shirt
302004         Clothing          Nike       Shorts
302007         Clothing        Adidas       Jacket

[54139 rows x 3 columns]

Number of entries corrected: 59


In [10]:
retaildf.iloc[0:5, 0:20]

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount
0,8691788,37249,Michelle Harrington,Ebony39@gmail.com,1414786801,3959 Amanda Burgs,Dortmund,Berlin,77985,Germany,21,Male,Low,Regular,2023-09-18,2023,September,22:03:55,3,108.03
1,2174773,69749,Kelsey Hill,Mark36@gmail.com,6852899987,82072 Dawn Centers,Nottingham,England,99071,UK,19,Female,Low,Premium,2023-12-31,2023,December,8:42:04,2,403.35
2,6679610,30192,Scott Jensen,Shane85@gmail.com,8362160449,4133 Young Canyon,Geelong,New South Wales,75929,Australia,48,Male,Low,Regular,2023-04-26,2023,April,4:06:29,3,354.48
3,7232460,62101,Joseph Miller,Mary34@gmail.com,2776751724,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420,Canada,56,Male,High,Premium,2023-05-08,2023,May,14:55:17,7,352.41
4,4983775,27901,Debra Coleman,Charles30@gmail.com,9098267635,5813 Lori Ports Suite 269,Bristol,England,48704,UK,22,Male,Low,Premium,2024-01-10,2024,January,16:54:07,2,124.28


In [11]:
retaildf.iloc[0:5, 20:30]

,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5,Cycling shorts
1,806.71,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4,Lenovo Tab
2,1063.43,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2,Sports equipment
3,2466.85,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4,Utility knife
4,248.55,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1,Chocolate cookies


Now that we have corrected the data - handled missing values, we will prepare it for high-level analysis and proper dashboarding!
Steps involve: 

1. Filter based on Country and Product Category - so we will create a subset version of the dataset!
2. Edit the dataset to incorporate both transaction-based and customer-based approaches.

In [12]:
clothing_df = retaildf[retaildf['Product_Category'] == 'Clothing']
clothing_df.iloc[0:6, 0:20]

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Date,Year,Month,Time,Total_Purchases,Amount
0,8691788,37249,Michelle Harrington,Ebony39@gmail.com,1414786801,3959 Amanda Burgs,Dortmund,Berlin,77985,Germany,21,Male,Low,Regular,2023-09-18,2023,September,22:03:55,3,108.03
7,2344675,26603,Angela Fields,Tanya94@gmail.com,3668096144,237 Young Curve,Munich,Berlin,86862,Germany,29,Male,Medium,Premium,2023-03-24,2023,March,10:12:56,1,46.59
14,2401331,98300,Andrew Guzman,Eric76@gmail.com,2923044936,470 Rodriguez Estate Suite 564,Portsmouth,England,5259,UK,32,Female,High,Regular,2023-11-18,2023,November,23:41:05,6,297.73
30,3413296,98256,Steven Boyd,Sheryl18@gmail.com,5248581519,1719 Christina Heights Apt. 614,Portsmouth,England,43602,UK,26,Female,Low,Regular,2024-02-06,2024,February,1:59:20,3,159.80
36,8783109,65305,Jonathan Jenkins,Christina19@gmail.com,8524180606,890 Brooks Pine,Portsmouth,England,68300,UK,19,Male,High,Regular,2023-12-12,2023,December,17:48:31,5,195.55
39,2458233,96840,Susan Thomas,Bonnie80@gmail.com,9540868010,48415 Ferguson Passage,Portsmouth,England,85074,UK,65,Male,Low,Regular,2023-10-29,2023,October,13:43:05,8,28.83


In [13]:
clothing_df.iloc[0:6, 20:31]
#clothing_df.isnull().sum() #no null values

,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5,Cycling shorts
7,46.59,Clothing,Zara,Shirt,Bad,Same-Day,Cash,Processing,1,Dress shirt
14,1786.36,Clothing,Adidas,T-shirt,Bad,Same-Day,Cash,Shipped,1,V-neck tee
30,479.41,Clothing,Adidas,Shoes,Good,Same-Day,Debit Card,Pending,3,Running shoes
36,977.76,Clothing,Zara,Shirt,Good,Express,Credit Card,Pending,4,Henley shirt
39,230.60,Clothing,Adidas,Shoes,Excellent,Same-Day,Credit Card,Pending,4,Running shoes


The dataset has been filtered to only include the clothing product category! Now let's prepare a clean dataset!

In [14]:
columns_to_drop = ['Name', 'Email', 'Phone', 'Address', 'Zipcode']
clothing_df = clothing_df.drop(columns=columns_to_drop)

# Select relevant columns
relevant_columns = [
    'Transaction_ID', 'Customer_ID', 'Age', 'Gender', 'Income', 'Customer_Segment',
    'Date', 'Year', 'Month', 'Total_Purchases', 'Amount', 'Product_Category',
    'Product_Brand', 'Product_Type', 'Ratings', 'Feedback', 'Shipping_Method', 
    'Payment_Method', 'Order_Status','products', 'City', 'State','Country'
]

clothing_df = clothing_df[relevant_columns]


In [15]:
duplicate_transactions = clothing_df[clothing_df.duplicated(subset='Transaction_ID', keep=False)]

# Display the duplicate transactions
#print(duplicate_transactions) 
#1113 duplicate transactions!

In [16]:
# Assuming your DataFrame is called 'clothing_df'
duplicates = clothing_df[clothing_df.duplicated(subset=['Transaction_ID'], keep=False)]
grouped = duplicates.groupby('Transaction_ID')

# Counter to limit the number of cases
counter = 0
max_cases = 3

for transaction_id, group in grouped:
    if counter >= max_cases:
        break
    print(f"\n## Transaction ID: {transaction_id}")
    print(group.to_string(index=False))
    print("\n" + "-"*80)
    counter += 1


## Transaction ID: 1031034
 Transaction_ID  Customer_ID  Age Gender Income Customer_Segment       Date  Year Month  Total_Purchases  Amount Product_Category Product_Brand Product_Type  Ratings  Feedback Shipping_Method Payment_Method Order_Status       products       City   State Country
        1031034        30303   45 Female Medium          Regular 2023-07-02  2023  July                5  182.04         Clothing        Adidas       Jacket        3      Good         Express    Credit Card   Processing        Peacoat Nottingham England      UK
        1031034        30303   45 Female Medium          Regular 2023-07-02  2023  July                5  182.04         Clothing        Adidas       Jacket        4 Excellent         Express           Cash   Processing Varsity jacket Nottingham England      UK

--------------------------------------------------------------------------------

## Transaction ID: 1081053
 Transaction_ID  Customer_ID  Age Gender Income Customer_Segment       Date 

In [17]:
#Resolving the issue of duplicate Transaction IDs
duplicates = clothing_df[clothing_df.duplicated('Transaction_ID', keep=False)]

# Display duplicates
print("Duplicates:")
print(duplicates.shape)

# Resolve duplicates by keeping the first occurrence (customize as needed)
clothing_df = clothing_df.drop_duplicates(subset='Transaction_ID', keep='first')

# Verify no duplicates remain
assert clothing_df['Transaction_ID'].is_unique

print(f"Number of unique Transaction_IDs: {clothing_df['Transaction_ID'].nunique()}")
print(f"Total number of rows: {len(clothing_df)}")

Duplicates:
(1113, 23)
Number of unique Transaction_IDs: 53582
Total number of rows: 53582


The dataset has unique transaction IDs!

In [18]:
# Group the dataframe by 'Customer_ID' and then by 'Transaction_ID'
grouped_df = clothing_df.groupby(['Customer_ID', 'Transaction_ID'])

# To see the transactions a customer has made, you can iterate over the groups or display a sample
# Let's display a few rows from each group as an example
grouped_sample = grouped_df.first()

# Display the first few rows of the grouped data
grouped_sample.head(10)
#this datase has data integrity issues- for eg- multiple people have been given same customer ID!!! 

Age  Gender  Income Customer_Segment       Date  \
Customer_ID Transaction_ID                                                    
10000       4340470          26    Male    High          Regular 2023-11-19   
10001       5025854          46  Female     Low          Regular 2023-10-09   
10003       4518045          24  Female  Medium          Regular 2023-07-17   
10006       1479952          23  Female    High          Regular 2023-07-11   
            3934671          23  Female    High          Regular 2024-02-14   
            5262562          51  Female  Medium          Regular 2024-01-16   
            8422912          46    Male     Low          Premium 2023-10-11   
10007       4896527          19    Male     Low              New 2024-01-02   
            6812749          35  Female  Medium          Regular 2023-05-30   
10008       5833556          26  Female    High          Regular 2024-02-06   

                            Year     Month  Total_Purchases  Amount  \
Customer_ID Transaction_ID                                            
10000       4340470         2023   January                5  299.74   
10001       5025854         2023   October                8  484.94   
10003       4518045         2023    August                9  210.08   
10006       1479952         2023      July                7   69.11   
            3934671         2024  February               10  338.32   
            5262562         2024   January                6  467.92   
            8422912         2023   October                3  146.98   
10007       4896527         2024   January                7  281.19   
            6812749         2023      July               10   90.04   
10008       5833556         2024   January                1  441.91   

                           Product_Category  ... Product_Type Ratings  \
Customer_ID Transaction_ID                   ...                        
10000       4340470                Clothing  ...       Shorts       3   
10001       5025854                Clothing  ...        Shirt       5   
10003       4518045                Clothing  ...        Shoes       2   
10006       1479952                Clothing  ...      T-shirt       4   
            3934671                Clothing  ...       Jacket       1   
            5262562                Clothing  ...       Jacket       2   
            8422912                Clothing  ...        Shoes       5   
10007       4896527                Clothing  ...        Shirt       4   
            6812749                Clothing  ...      T-shirt       4   
10008       5833556                Clothing  ...        Shoes       1   

                             Feedback Shipping_Method Payment_Method  \
Customer_ID Transaction_ID                                             
10000       4340470              Good        Same-Day           Cash   
10001       5025854         Excellent        Same-Day           Cash   
10003       4518045           Average        Standard     Debit Card   
10006       1479952              Good        Standard         PayPal   
            3934671               Bad        Same-Day     Debit Card   
            5262562           Average        Standard         PayPal   
            8422912         Excellent        Same-Day         PayPal   
10007       4896527         Excellent         Express    Credit Card   
            6812749         Excellent         Express    Credit Card   
10008       5833556               Bad        Same-Day           Cash   

                           Order_Status       products           City  \
Customer_ID Transaction_ID                                              
10000       4340470           Delivered   Khaki shorts  San Francisco   
10001       5025854          Processing       Tank top        Hanover   
10003       4518045           Delivered     High heels        Raleigh   
10006       1479952           Delivered      Plain tee         Boston   
            3934671             Shipped       

In [19]:
# Group by 'Customer_ID' and count unique 'Transaction_ID's for each customer
transaction_counts = clothing_df.groupby('Customer_ID')['Transaction_ID'].nunique()

# Filter customers who have more than one transaction
customers_with_multiple_transactions = transaction_counts[transaction_counts > 1]

# Print the customer IDs and their transaction counts
customers_with_multiple_transactions
# so there are 10764 customers who have made multiple transactions

Customer_ID
10006    4
10007    2
10010    2
10015    2
10022    3
        ..
99952    2
99962    2
99990    2
99994    2
99996    2
Name: Transaction_ID, Length: 10727, dtype: int64

In [20]:
# Count the occurrences of each Customer_ID
customer_id_counts = clothing_df['Customer_ID'].value_counts()

# Find Customer_IDs that appear more than once
duplicate_customer_ids = customer_id_counts[customer_id_counts > 1]

# Display duplicate Customer_IDs and their counts
print(duplicate_customer_ids)

70915    7
79803    6
88982    6
71236    5
94824    5
        ..
92107    2
78203    2
58718    2
47063    2
90907    2
Name: Customer_ID, Length: 10727, dtype: int64


In [21]:
clothing_df.iloc[0:7,0:10]

,Transaction_ID,Customer_ID,Age,Gender,Income,Customer_Segment,Date,Year,Month,Total_Purchases
0,8691788,37249,21,Male,Low,Regular,2023-09-18,2023,September,3
7,2344675,26603,29,Male,Medium,Premium,2023-03-24,2023,March,1
14,2401331,98300,32,Female,High,Regular,2023-11-18,2023,November,6
30,3413296,98256,26,Female,Low,Regular,2024-02-06,2024,February,3
36,8783109,65305,19,Male,High,Regular,2023-12-12,2023,December,5
39,2458233,96840,65,Male,Low,Regular,2023-10-29,2023,October,8
44,3239367,62464,58,Female,High,Regular,2023-12-14,2023,December,3


In [22]:
def mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None


In [23]:
# Group by 'New_Customer_ID'
grouped = clothing_df.groupby('Customer_ID')

# Get the most common gender and customer segment
most_common_gender = grouped['Gender'].apply(mode)
most_common_segment = grouped['Customer_Segment'].apply(mode)

# Get the most recent age and income (using Transaction_Date for ordering if available)
# Assuming 'Transaction_Date' exists and is of datetime type
most_recent_age = grouped.apply(lambda x: x.sort_values(by='Date').iloc[-1]['Age'])
most_recent_income = grouped.apply(lambda x: x.sort_values(by='Date').iloc[-1]['Income'])

# Compile the results into a DataFrame
updated_info = pd.DataFrame({
    'Customer_ID': most_common_gender.index,
    'Most_Common_Gender': most_common_gender.values,
    'Most_Common_Segment': most_common_segment.values,
    'Most_Recent_Age': most_recent_age.values,
    'Most_Recent_Income': most_recent_income.values
})

print(updated_info.head())


   Customer_ID Most_Common_Gender Most_Common_Segment  Most_Recent_Age  \
0        10000               Male             Regular               26   
1        10001             Female             Regular               46   
2        10003             Female             Regular               24   
3        10006             Female             Regular               23   
4        10007             Female                 New               19   

  Most_Recent_Income  
0               High  
1                Low  
2             Medium  
3               High  
4                Low  


In [24]:
# Merge the updated information back to the original dataframe
clothing_df_updated = clothing_df.merge(updated_info, on='Customer_ID', suffixes=('', '_Updated'))

# Replace the original columns with updated values
clothing_df_updated['Gender'] = clothing_df_updated['Most_Common_Gender']
clothing_df_updated['Customer_Segment'] = clothing_df_updated['Most_Common_Segment']
clothing_df_updated['Age'] = clothing_df_updated['Most_Recent_Age']
clothing_df_updated['Income'] = clothing_df_updated['Most_Recent_Income']

# Drop the helper columns used for merging
clothing_df_updated.drop(columns=['Most_Common_Gender', 'Most_Common_Segment', 'Most_Recent_Age', 'Most_Recent_Income'], inplace=True)

print(clothing_df_updated.head())


   Transaction_ID  Customer_ID  Age  Gender Income Customer_Segment  \
0         8691788        37249   46    Male    Low              New   
1         4895457        37249   46    Male    Low              New   
2         2344675        26603   26  Female   High          Regular   
3         6774063        26603   26  Female   High          Regular   
4         7526592        26603   26  Female   High          Regular   

        Date  Year      Month  Total_Purchases  ...  Product_Type Ratings  \
0 2023-09-18  2023  September                3  ...        Shorts       5   
1 2023-12-08  2023   December                4  ...         Shirt       5   
2 2023-03-24  2023      March                1  ...         Shirt       1   
3 2023-05-24  2023        May                3  ...       T-shirt       3   
4 2023-08-30  2023    January               10  ...         Shoes       5   

    Feedback Shipping_Method  Payment_Method Order_Status        products  \
0  Excellent        Same-Day     

In [25]:
import pandas as pd

# Sample DataFrame (replace with your actual DataFrame)
# clothing_df = pd.read_csv('your_data.csv')

# Step 1: Define age groups
def age_group(age):
    if pd.isna(age):  # Handle missing values
        return 'Unknown'
    if age < 18:
        return 'Under 18'
    elif 18 <= age <= 25:
        return '18-25'
    elif 26 <= age <= 35:
        return '26-35'
    elif 36 <= age <= 45:
        return '36-45'
    elif 46 <= age <= 55:
        return '46-55'
    elif 56 <= age <= 65:
        return '56-65'
    else:
        return '65+'

clothing_df_updated['Age_Group'] = clothing_df_updated['Age'].apply(age_group)

age_distribution = clothing_df_updated['Age_Group'].value_counts().sort_index()

# Print the distribution
print("Age Group Distribution:")
print(age_distribution)

# Step 2: Define generation based on birth year
def get_generation(age):
    if pd.isna(age):  # Handle missing values
        return 'Unknown'
    current_year = pd.Timestamp.now().year
    birth_year = current_year - age
    
    if 2010 <= birth_year:
        return 'Generation Alpha'
    elif 1997 <= birth_year <= 2009:
        return 'Gen Z'
    elif 1981 <= birth_year <= 1996:
        return 'Millennial'
    elif 1965 <= birth_year <= 1980:
        return 'Gen X'
    elif 1946 <= birth_year <= 1964:
        return 'Boomer'
    elif 1928 <= birth_year <= 1945:
        return 'Silent Generation'
    else:
        return 'Greatest Generation'

clothing_df_updated['Generation'] = clothing_df_updated['Age'].apply(get_generation)

generation_distribution = clothing_df_updated['Generation'].value_counts().sort_index()

# Print the generation distribution
print("Generation Distribution:")
print(generation_distribution)


Age Group Distribution:
18-25    21853
26-35     9048
36-45     4733
46-55    10661
56-65     4845
65+       2442
Name: Age_Group, dtype: int64
Generation Distribution:
Boomer         5348
Gen X         13522
Gen Z         25645
Millennial     9067
Name: Generation, dtype: int64


In [26]:
print(clothing_df_updated.head())

   Transaction_ID  Customer_ID  Age  Gender Income Customer_Segment  \
0         8691788        37249   46    Male    Low              New   
1         4895457        37249   46    Male    Low              New   
2         2344675        26603   26  Female   High          Regular   
3         6774063        26603   26  Female   High          Regular   
4         7526592        26603   26  Female   High          Regular   

        Date  Year      Month  Total_Purchases  ...   Feedback  \
0 2023-09-18  2023  September                3  ...  Excellent   
1 2023-12-08  2023   December                4  ...  Excellent   
2 2023-03-24  2023      March                1  ...        Bad   
3 2023-05-24  2023        May                3  ...       Good   
4 2023-08-30  2023    January               10  ...  Excellent   

  Shipping_Method Payment_Method Order_Status        products            City  \
0        Same-Day     Debit Card      Shipped  Cycling shorts        Dortmund   
1         Expr

## Country- 
array(['Germany', 'Canada', 'Australia', 'USA', 'UK'], dtype=object)

## State-
array(['Berlin', 'Ontario', 'New South Wales', 'Maine', 'England',
       'West Virginia', 'South Carolina', 'California', 'Connecticut',
       'Texas', 'Idaho', 'Louisiana', 'Nevada', 'Utah', 'Delaware',
       'Georgia', 'Kansas', 'Ohio', 'Minnesota', 'North Dakota',
       'Indiana', 'Washington', 'New Mexico', 'South Dakota', 'Michigan',
       'Illinois', 'North Carolina', 'Oklahoma', 'Missouri', 'New York',
       'Mississippi', 'Tennessee', 'Nebraska', 'New Jersey', 'Arkansas',
       'Florida', 'Massachusetts', 'Vermont', 'Maryland', 'Kentucky',
       'Alaska', 'Alabama', 'Arizona', 'Colorado', 'Rhode Island',
       'Virginia', 'Pennsylvania', 'Oregon', 'Montana', 'Wyoming',
       'New Hampshire', 'Wisconsin', 'Hawaii', 'Iowa'], dtype=object)
       
## City - 
array(['Dortmund', 'Hamilton', 'Munich', 'Albury-Wodonga',
       'San Francisco', 'Portsmouth', 'Toowoomba', 'Calgary', 'Raleigh',
       'Colorado Springs', 'Kitchener', 'Oakland', 'Chicago', 'Frankfurt',
       'Manchester', 'Columbus', 'Montreal', 'London', 'Melbourne',
       'Washington', 'Charlotte', 'Sydney', 'Minneapolis', 'Düsseldorf',
       'Cologne', 'Stuttgart', 'Belfast', 'Nottingham', 'San Diego',
       'Windsor', 'Oshawa', 'Brisbane', 'Edinburgh', 'Wuppertal',
       'Canberra', 'Newcastle upon Tyne', 'Bonn', 'Hull', 'Cairns',
       'Leicester', 'Bendigo', 'Nuremberg', 'Berlin', 'Duisburg',
       'Dresden', 'Southampton', "St. John's", 'Hamburg', 'Barrie',
       'Oxford', 'Boston', 'New York', 'Long Beach', 'Hobart',
       'Townsville', 'Darwin', 'Birmingham', 'Wollongong', 'Newcastle',
       'Bremen', 'Austin', 'Ballarat', 'Ottawa', 'Mackay', 'Geelong',
       'Bochum', 'Leeds', 'Edmonton', 'Adelaide', 'Cleveland',
       'Fort Worth', 'Bielefeld', 'Münster', 'Sheffield', 'Winnipeg',
       'Launceston', 'Brighton', 'Perth', 'New Orleans', 'Liverpool',
       'Seattle', 'Atlanta', 'San Antonio', 'Cardiff', 'Halifax',
       'Louisville', 'Mesa', 'Quebec City', 'Saskatoon', 'Virginia Beach',
       'Houston', 'Philadelphia', 'Plymouth', 'Toronto', 'Leipzig',
       'Gold Coast', 'Kansas City', 'Victoria', 'El Paso', 'Regina',
       'Vancouver', 'Miami', 'Tulsa', 'Oklahoma City', 'Kelowna',
       'Hanover', 'Denver', 'Las Vegas', 'San Jose', 'Phoenix',
       'Nashville', 'Indianapolis', 'Tucson', 'Los Angeles', 'Essen',
       'Fresno', 'Jacksonville', 'Arlington', 'Glasgow', 'Portland',
       'Baltimore', 'Sacramento', 'Memphis', 'Albuquerque', 'Detroit',
       'Wichita', 'Omaha', 'Bristol', 'Dallas', 'Milwaukee'], dtype=object)

In [41]:
import pandas as pd

# Provided mapping
country_state_city_mapping = {
    "Germany": {
        "Berlin": ["Berlin"],
        "North Rhine-Westphalia": ["Dortmund", "Düsseldorf", "Cologne", "Wuppertal", "Bonn", "Essen", "Bochum", "Bielefeld", "Münster"],
        "Bavaria": ["Munich", "Nuremberg"],
        "Hesse": ["Frankfurt"],
        "Bremen": ["Bremen"],
        "Hamburg": ["Hamburg"],
        "Saxony": ["Dresden", "Leipzig"]
    },
    "Canada": {
        "Ontario": ["Hamilton", "Kitchener", "Toronto", "Ottawa", "Windsor", "Oshawa", "London", "Barrie"],
        "Alberta": ["Calgary", "Edmonton"],
        "Quebec": ["Montreal", "Quebec City"],
        "British Columbia": ["Vancouver", "Victoria"],
        "Manitoba": ["Winnipeg"],
        "Nova Scotia": ["Halifax"],
        "Saskatchewan": ["Saskatoon", "Regina"],
        "Newfoundland and Labrador": ["St. John's"],
        "New Brunswick": ["Moncton"]
    },
    "Australia": {
        "New South Wales": ["Sydney", "Newcastle", "Wollongong"],
        "Victoria": ["Melbourne", "Geelong", "Ballarat"],
        "Queensland": ["Brisbane", "Gold Coast", "Townsville", "Cairns", "Mackay", "Toowoomba"],
        "South Australia": ["Adelaide"],
        "Western Australia": ["Perth"],
        "Tasmania": ["Hobart", "Launceston"],
        "Northern Territory": ["Darwin"],
        "Australian Capital Territory": ["Canberra"]
    },
    "USA": {
        "Maine": [],
        "West Virginia": [],
        "South Carolina": [],
        "California": ["San Francisco", "Oakland", "San Diego", "Los Angeles", "Long Beach", "San Jose", "Sacramento", "Fresno"],
        "Connecticut": [],
        "Texas": ["Austin", "Houston", "Dallas", "San Antonio", "Fort Worth", "El Paso", "Arlington"],
        "Idaho": [],
        "Louisiana": ["New Orleans"],
        "Nevada": ["Las Vegas"],
        "Utah": ["Salt Lake City"],
        "Delaware": [],
        "Georgia": ["Atlanta"],
        "Kansas": ["Wichita", "Kansas City"],
        "Ohio": ["Cleveland", "Columbus"],
        "Minnesota": ["Minneapolis"],
        "North Dakota": [],
        "Indiana": ["Indianapolis"],
        "Washington": ["Seattle"],
        "New Mexico": ["Albuquerque"],
        "South Dakota": [],
        "Michigan": ["Detroit"],
        "Illinois": ["Chicago"],
        "North Carolina": ["Raleigh", "Charlotte"],
        "Oklahoma": ["Oklahoma City", "Tulsa"],
        "Missouri": ["Kansas City"],
        "New York": ["New York City"],
        "Mississippi": [],
        "Tennessee": ["Nashville", "Memphis"],
        "Nebraska": ["Omaha"],
        "New Jersey": [],
        "Arkansas": [],
        "Florida": ["Miami", "Jacksonville"],
        "Massachusetts": ["Boston"],
        "Vermont": [],
        "Maryland": ["Baltimore"],
        "Kentucky": ["Louisville"],
        "Alaska": [],
        "Alabama": [],
        "Arizona": ["Phoenix", "Mesa", "Tucson"],
        "Colorado": ["Denver", "Colorado Springs"],
        "Rhode Island": [],
        "Virginia": ["Virginia Beach"],
        "Pennsylvania": ["Philadelphia"],
        "Oregon": ["Portland"],
        "Montana": [],
        "Wyoming": [],
        "New Hampshire": [],
        "Wisconsin": ["Milwaukee"],
        "Hawaii": [],
        "Iowa": []
    },
    "UK": {
        "England": ["London", "Manchester", "Birmingham", "Leeds", "Sheffield", "Liverpool", "Nottingham", "Bristol", "Southampton", "Portsmouth", "Brighton", "Plymouth", "Hull", "Leicester", "Oxford"],
        "Scotland": ["Glasgow", "Edinburgh"],
        "Wales": ["Cardiff"],
        "Northern Ireland": ["Belfast"]
    }
}

# Convert the mapping to a DataFrame
mapping_list = []
for country, states in country_state_city_mapping.items():
    for state, cities in states.items():
        for city in cities:
            mapping_list.append([country, state, city])

mapping_df = pd.DataFrame(mapping_list, columns=['Country', 'State', 'City'])
print(mapping_df.head())


   Country                   State        City
0  Germany                  Berlin      Berlin
1  Germany  North Rhine-Westphalia    Dortmund
2  Germany  North Rhine-Westphalia  Düsseldorf
3  Germany  North Rhine-Westphalia     Cologne
4  Germany  North Rhine-Westphalia   Wuppertal


In [43]:
# Assuming your original DataFrame is named 'df'
# Merge the original DataFrame with the mapping DataFrame
clothing_df_updated = clothing_df_updated.merge(mapping_df, on='City', how='left', suffixes=('_old', ''))

# Replace incorrect values with correct ones
clothing_df_updated['State'] = clothing_df_updated['State'].fillna(clothing_df_updated['State_old'])
clothing_df_updated['Country'] = clothing_df_updated['Country'].fillna(clothing_df_updated['Country_old'])

# Drop the old columns
clothing_df_updated = clothing_df_updated.drop(['State_old', 'Country_old'], axis=1)

print(clothing_df_updated.sample(10))

       Transaction_ID  Customer_ID  Age  Gender  Income Customer_Segment  \
8762          4339132        17674   22  Female  Medium              New   
11776         1237183        29042   22  Female  Medium          Regular   
51123         1943568        54015   20  Female    High              New   
4274          8659070        86920   19    Male  Medium              New   
35648         4145417        83409   23    Male    High          Regular   
19164         6074825        17676   20    Male  Medium              New   
50352         8661279        10838   48    Male     Low          Premium   
32741         7204073        49311   26  Female    High          Regular   
21892         5487877        12923   34    Male  Medium          Premium   
25799         4317916        14007   46  Female    High              New   

            Date  Year      Month  Total_Purchases  ...   Feedback  \
8762  2023-07-01  2023       July                1  ...       Good   
11776 2023-11-19  2023 

In [45]:
#missing incorrect mappings
# Identify rows where mapping is still incorrect or missing
missing_mappings = clothing_df_updated[clothing_df_updated['Country'].isnull() | clothing_df_updated['State'].isnull()]

# Print these for manual review
print(missing_mappings)



Empty DataFrame
Columns: [Transaction_ID, Customer_ID, Age, Gender, Income, Customer_Segment, Date, Year, Month, Total_Purchases, Amount, Product_Category, Product_Brand, Product_Type, Ratings, Feedback, Shipping_Method, Payment_Method, Order_Status, products, City, Age_Group, Generation, Country, State]
Index: []

[0 rows x 25 columns]


## PAUSE!

In [46]:

# 1. Products DataFrame
products_df = clothing_df_updated[['Product_Category', 'Product_Brand', 'Product_Type', 'products']].drop_duplicates().reset_index(drop=True)
products_df['Product_ID'] = products_df.index + 1  # Create a unique Product_ID
products_df = products_df[['Product_ID', 'Product_Category', 'Product_Brand', 'Product_Type', 'products']]

# 2. Customers DataFrame
customers_df = clothing_df_updated[['Customer_ID', 'Age', 'Gender', 'Income', 'Customer_Segment', 'Age_Group', 'Generation', 'Country', 'State', 'City']].drop_duplicates().reset_index(drop=True)

# 3. Transactions DataFrame
transactions_df = clothing_df_updated[['Transaction_ID', 'Customer_ID', 'Date', 'Year', 'Month', 'Total_Purchases', 'Amount', 'Product_Category', 'Product_Brand', 'Product_Type', 'Ratings', 'Feedback', 'Country', 'State', 'City']]
# Add Product_ID to transactions
transactions_df = transactions_df.merge(products_df, on=['Product_Category', 'Product_Brand', 'Product_Type'], how='left')
transactions_df = transactions_df.drop(['Product_Category', 'Product_Brand', 'Product_Type', 'products'], axis=1)

# 4. Order Status DataFrame
order_status_df = clothing_df_updated[['Transaction_ID', 'Order_Status', 'Shipping_Method', 'Payment_Method']].drop_duplicates().reset_index(drop=True)

# 5. Customer Purchase Behavior DataFrame
customer_behavior_df = clothing_df_updated.groupby(['Customer_ID', 'Country', 'State', 'City']).agg({
    'Transaction_ID': 'count',
    'Amount': 'sum',
    'Total_Purchases': 'mean'
}).reset_index()
customer_behavior_df.rename(columns={'Transaction_ID': 'Total_Transactions', 'Amount': 'Total_Spent', 'Total_Purchases': 'Average_Purchases'}, inplace=True)


In [47]:
# Displaying the results
print("Products DataFrame:")
print(products_df.head())

print("\nCustomers DataFrame:")
print(customers_df.head())

print("\nTransactions DataFrame:")
print(transactions_df.sample(20))

print("\nOrder Status DataFrame:")
print(order_status_df.head())

print("\nCustomer Purchase Behavior DataFrame:")
print(customer_behavior_df.head())

Products DataFrame:
   Product_ID Product_Category Product_Brand Product_Type        products
0           1         Clothing          Nike       Shorts  Cycling shorts
1           2         Clothing          Zara        Shirt      Polo shirt
2           3         Clothing          Zara        Shirt     Dress shirt
3           4         Clothing          Nike      T-shirt  Scoop neck tee
4           5         Clothing          Nike        Shoes        Slippers

Customers DataFrame:
   Customer_ID  Age  Gender Income Customer_Segment Age_Group Generation  \
0        37249   46    Male    Low              New     46-55      Gen X   
1        37249   46    Male    Low              New     46-55      Gen X   
2        26603   26  Female   High          Regular     26-35      Gen Z   
3        26603   26  Female   High          Regular     26-35      Gen Z   
4        26603   26  Female   High          Regular     26-35      Gen Z   

     Country                   State            City  
0 

# Dataset Ready for Further Exploration - Analysis and VIZ!

#Saving DataFrames to CSV files
products_df.to_csv('/Users/snehakarki/Desktop/clothing/products.csv', index=False)
customers_df.to_csv('/Users/snehakarki/Desktop/clothing/customers.csv', index=False)
transactions_df.to_csv('/Users/snehakarki/Desktop/clothing/transactions.csv', index=False)
order_status_df.to_csv('/Users/snehakarki/Desktop/clothing/order_status.csv', index=False)
customer_behavior_df.to_csv('/Users/snehakarki/Desktop/clothing/customer_behavior.csv', index=False)

